# PRACTICA 6 PRUEBA CLUSTER

In [1]:
import redis

Se crean diferentes instancias, cada una de ellas conectada a un servidor del cluster

In [6]:
r1 = redis.Redis(host='127.0.0.1', port=7000)
r2 = redis.Redis(host='127.0.0.1', port=7002)
r3 = redis.Redis(host='127.0.0.1', port=7004)

Se inserta un valor en la instacia 1 del servidor, se espera que falle puesto que el valor que se introduce no pertenece al slot asignado al servidor 1

In [8]:
r1.set('Raul', 'mola')
r1.get('Raul')

ResponseError: CLUSTERDOWN The cluster is down

Se prueba forma alternativa para comprobar que con trys y catches se puede realizar cualquier consulta

In [ ]:
try:
    r1.set('Raul', 'mola')
    print("Se ha introducido en el nodo 1")
except:
    try:
        r2.set('Raul', 'mola')
        print("Se ha introducido en el nodo 2")
    except:
        try:
            r3.set('Raul','mola')
            print("Se ha introducido en el nodo 3")
        except:
            print("couldnt complete the set opertation")

Se prueba nueva forma de acceder al cluster. De esta forma se calcula de forma automatica a que nodo pertenece cada string introducido

In [ ]:
from rediscluster import RedisCluster

Se conecta a los nodos del cluster

startup_nodes = [{"host": "127.0.0.1", "port": "7000"},{"host": "127.0.0.1", "port": "7002"},{"host": "127.0.0.1", "port": "7004"}]
rc = RedisCluster(startup_nodes=startup_nodes, decode_responses=True)

Se obtiene el slot determinado para una clave concreta

In [ ]:
rc.cluster_keyslot("foo")

Debido a la lentitud de método anterior, se va a explorar una forma alternativa más optimizada

In [ ]:
import crcmod
#Variable que contiene los slots en función del nodo
SlotToServer=dict([(i,r1) for i in range(5461)]+
     [(j,r2) for j in range(5461,10923)]+
     [(k,r3) for k in range(10923,16384)])


def getSlot(key):
    #key must be binary
    crc16 = crcmod.mkCrcFun(0x11021, rev=False, initCrc=0x0000, xorOut=0x0000)
    return crc16(key) % 16384

Se carga una clave y un valor en la base de datos

In [ ]:
r=SlotToServer[getSlot(b'foo')]
r.set('foo','bar')